In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\monkeydc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
raw_data = pd.read_csv('./dataset.csv', sep=',')

# Rerating review AND data preparing

In [3]:
for index in range(len(raw_data)):
    if raw_data.at[index, 'star_rating'] > 3:
        raw_data.at[index, 'star_rating'] = 1
    elif raw_data.at[index, 'star_rating'] == 3:
        raw_data.at[index, 'star_rating'] = 2
    else:
        raw_data.at[index, 'star_rating'] = 0

In [4]:
from bs4 import BeautifulSoup
import re
# data cleaning
raw_data['review_body'] = raw_data['review_body'].str.lower()
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: re.sub(r'http\S+', '', BeautifulSoup(x, 'html.parser').get_text()))
def remove_non_alphabetical(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

raw_data['review_body'] = raw_data['review_body'].apply(remove_non_alphabetical)
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: ' '.join(x.split()))
contraction_dict = contraction_dict = {
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "you're": "you are",
    "you've": "you have",
    "you'll": "you will",
    "when's": "when is",
    "let's": "let us",
    "'cause": "because",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "couldn't": "could not",
    "wasn't": "was not",
    "weren't": "were not",
    "I'm": "I am",
    "I've": "I have",
    "I'll": "I will",
    "it's": "it is",
    "that's": "that is",
    "who's": "who is",
    "what's": "what is",
    "where's": "where is",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "she's": "she is",
    "he's": "he is",
    "how's": "how is",
    "you'd": "you would",
    "we'd": "we would",
    "they'd": "they would",
}
raw_data['review_body'] = raw_data['review_body'].replace(contraction_dict, regex=True)

C:\Users\monkeydc\.conda\envs\561\lib\site-packages\ipykernel_launcher.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  """
C:\Users\monkeydc\.conda\envs\561\lib\site-packages\ipykernel_launcher.py:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  """


In [5]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    words = text.split(" ")
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
raw_data['review_body'] = raw_data['review_body'].apply(remove_stop_words)
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4') # I have to download this dataset
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\monkeydc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\monkeydc\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

# Need to import String 

In [7]:
# train on personal dataset
import string
def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english') + list(string.punctuation))
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens
raw_data['clean_reviw'] = raw_data['review_body'].apply(preprocess_text)
# better word2vec
# model = Word2Vec(sentences=raw_data['clean_reviw'], vector_size=300, window=5, min_count=1, workers=4)

# train self W2V and Save model(named word2vec_model_b.txt)

In [8]:
model = Word2Vec(sentences=raw_data['clean_reviw'], vector_size=300, window=11, min_count=10, workers=4)

In [9]:
model.wv.save_word2vec_format('word2vec_model_b.txt', binary=False)

In [10]:
# every individual jupyter notebook need to use this .txt to reload self W2V

# Compare google pretrain and self-train W2V in word similarity

In [11]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('word2vec_model_b.txt', binary=False)

In [12]:
import numpy as np
def simple_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    similarity = dot_product / (norm_vector1 * norm_vector2)
    return similarity

In [18]:
print("Similariy between water and rock : ", simple_similarity(model['water'],model['rock']))
print("Similariy between ice and rock : ", simple_similarity(model['ice'],model['rock']))
print("Similariy between apple and iphone : ", simple_similarity(model['apple'],model['iphone']))
print("Similariy between apple and ipad : ", simple_similarity(model['apple'],model['ipad']))

Similariy between water and rock :  0.10122213
Similariy between ice and rock :  0.234708
Similariy between apple and iphone :  0.59669954
Similariy between apple and ipad :  0.55011463


In [14]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [15]:
# Word Embedding Similarity

In [17]:
# Example on word2vec-google-news-300
print("Similariy between water and rock : ", wv.similarity('water', 'rock'))
print("Similariy between ice and rock : ", wv.similarity('ice', 'rock'))
print("Similariy between apple and iphone : ", wv.similarity('apple','iphone'))
print("Similariy between apple and ipad : ", wv.similarity('apple','ipad'))

Similariy between water and rock :  0.1448098
Similariy between ice and rock :  0.26914403
Similariy between apple and iphone :  0.3199041
Similariy between apple and ipad :  0.35862675


# SVM & percepton

In [18]:
# binary dataset
values_to_stratify = [0,1]
def stratified_sample(group, n=100000):
    return group.sample(n=n, random_state=1)

binary_data= raw_data[raw_data['star_rating'].isin(values_to_stratify)].groupby('star_rating', group_keys=False).apply(stratified_sample)

In [19]:
def average_word2vec(text, model=None, vw=None):
    if model:
        word_vectors = [model[word] for word in text if word in model.key_to_index]
    if vw:
        word_vectors = [wv[word] for word in text if word in wv.key_to_index]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(300)

In [20]:
binary_data['word2vec_features_google'] = binary_data['clean_reviw'].apply(lambda x: average_word2vec(x, vw=wv))

In [22]:
binary_data['word2vec_features_self'] = binary_data['clean_reviw'].apply(lambda x: average_word2vec(x, model=model))

In [24]:
from sklearn.utils import shuffle
shuffled_dataset = shuffle(binary_data, random_state=42)

In [25]:
from sklearn.model_selection import train_test_split
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(shuffled_dataset['word2vec_features_self'].tolist(), shuffled_dataset['star_rating'], test_size=0.2, random_state=42)
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(shuffled_dataset['word2vec_features_google'].tolist(), shuffled_dataset['star_rating'], test_size=0.2, random_state=42)

# google + SVM/percepton

In [26]:
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
perceptron_model = Perceptron()
perceptron_model.fit(X_train_g, y_train_g)
y_pred_perceptron = perceptron_model.predict(X_test_g)
accuracy_perceptron_g = accuracy_score(y_test_g, y_pred_perceptron)

# Train SVM model
svm_model = LinearSVC()
svm_model.fit(X_train_g, y_train_g)
y_pred_svm = svm_model.predict(X_test_g)
accuracy_svm_g = accuracy_score(y_test_g, y_pred_svm)

In [27]:
print(accuracy_perceptron_g)
print(accuracy_svm_g)

0.7609
0.816


# self + SVM/percepton

In [28]:
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
perceptron_model = Perceptron()
perceptron_model.fit(X_train_s, y_train_s)
y_pred_perceptron = perceptron_model.predict(X_test_s)
accuracy_perceptron = accuracy_score(y_test_s, y_pred_perceptron)

# Train SVM model
svm_model = LinearSVC()
svm_model.fit(X_train_s, y_train_s)
y_pred_svm = svm_model.predict(X_test_s)
accuracy_svm = accuracy_score(y_test_s, y_pred_svm)
print(accuracy_perceptron)
print(accuracy_svm)

0.724825
0.84435


C:\Users\monkeydc\.conda\envs\561\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


# Useless code (remove to different .in)

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

C:\Users\monkeydc\.conda\envs\561\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
X_train_tensor_b_s = torch.FloatTensor(X_train_s)
y_train_tensor_b_s = torch.LongTensor(y_train_s.values)
X_test_tensor_b_s = torch.FloatTensor(X_test_s)
y_test_tensor_b_s = torch.LongTensor(y_test_s.values)

C:\Users\monkeydc\.conda\envs\561\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  """Entry point for launching an IPython kernel.


In [31]:
X_train_tensor_b_g = torch.FloatTensor(X_train_g)
y_train_tensor_b_g = torch.LongTensor(y_train_g.values)
X_test_tensor_b_g = torch.FloatTensor(X_test_g)
y_test_tensor_b_g = torch.LongTensor(y_test_g.values)

In [32]:
input_size = 300
hidden_size1 = 50
hidden_size2 = 10
output_size_binary = 2
output_size_ternary = 3

In [33]:
# model
class MyMLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MyMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [34]:
# binary

In [36]:
# google + FNN
mlp_binary = MyMLP(input_size, hidden_size1, hidden_size2, output_size_binary)
criterion = nn.CrossEntropyLoss()
optimizer_binary = optim.Adam(mlp_binary.parameters(), lr=0.001)
for epoch in range(1500):
    outputs = mlp_binary(X_train_tensor_b_s)
    loss = criterion(outputs, y_train_tensor_b_s)
    optimizer_binary.zero_grad()
    loss.backward()
    optimizer_binary.step()

In [37]:
with torch.no_grad():
    mlp_binary.eval()
    y_pred_binary_s = torch.argmax(mlp_binary(X_test_tensor_b_s), dim=1)
    accuracy_binary_s = torch.sum(y_pred_binary_s == y_test_tensor_b_s).item() / len(y_test_tensor_b_s)

In [38]:
print(accuracy_binary_s)

0.861575


In [39]:
# self + FNN
mlp_binary = MyMLP(input_size, hidden_size1, hidden_size2, output_size_binary)
criterion = nn.CrossEntropyLoss()
optimizer_binary = optim.Adam(mlp_binary.parameters(), lr=0.001)
for epoch in range(1500):
    outputs = mlp_binary(X_train_tensor_b_g)
    loss = criterion(outputs, y_train_tensor_b_g)
    optimizer_binary.zero_grad()
    loss.backward()
    optimizer_binary.step()
with torch.no_grad():
    mlp_binary.eval()
    y_pred_binary_g = torch.argmax(mlp_binary(X_test_tensor_b_g), dim=1)
    accuracy_binary_g = torch.sum(y_pred_binary_g == y_test_tensor_b_g).item() / len(y_test_tensor_b_g)
print(accuracy_binary_g)

0.83945


In [40]:
# self+CNN # different dimension with FNN